<a href="https://colab.research.google.com/github/prometricas/Abeba_Datos_Masivos/blob/main/CategoriaDeVideosMasVista.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1. Instalación de Java y Spark**

## **Instalar JDK de Java**

In [ ]:
!apt-get install -y openjdk-11-jdk-headless -qq > /dev/null

## **Instalar Spark**

In [ ]:
# Descargar Spark
!wget https://dlcdn.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz

--2026-01-13 13:36:30--  https://dlcdn.apache.org/spark/spark-3.5.7/spark-3.5.7-bin-hadoop3.tgz
Resolving dlcdn.apache.org (dlcdn.apache.org)... 151.101.2.132, 2a04:4e42::644
Connecting to dlcdn.apache.org (dlcdn.apache.org)|151.101.2.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 400914067 (382M) [application/x-gzip]
Saving to: ‘spark-3.5.7-bin-hadoop3.tgz’

spark-3.5.7-bin-had 100%[===================>] 382.34M   236MB/s    in 1.6s    

2026-01-13 13:36:34 (236 MB/s) - ‘spark-3.5.7-bin-hadoop3.tgz’ saved [400914067/400914067]



In [ ]:
# Descomprimir
!tar xf spark-3.5.7-bin-hadoop3.tgz

# Mostrar folder descomprimido
!ls /content

0302	  sample_data		   spark-3.5.7-bin-hadoop3.tgz
0302.zip  spark-3.5.7-bin-hadoop3


## **Definir variables de entorno**


In [ ]:
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-17-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.5.7-bin-hadoop3"

## **Instalar e Iniciar findspark**
Esto permite usar *PySpark* como una librería estándar en Python.

In [ ]:
!pip install findspark
import findspark
findspark.init()

# **2. Importar carpeta con datos**
Descarga carpeta, desde URL dada, y la descomprime.

In [ ]:
import requests
import os
from tqdm import tqdm     # Librería para la barra de progreso

# Limpio ejecuciones previas
NOMBRE_ZIP = "0302.zip"
DATASET = NOMBRE_ZIP.split(".")[0]
! rm -rf {NOMBRE_ZIP}
! rm -rf {DATASET}

# Configuración
URL_BASE = "https://netsg.cs.sfu.ca/youtubedata/"
URL_COMPLETA = URL_BASE + NOMBRE_ZIP

# Defino función de descarga
def descargar_con_progreso(url, nombre_archivo):
    # Iniciamos la conexión con stream=True para bajar por pedazos
    response = requests.get(url, stream=True)
    total_size_in_bytes = int(response.headers.get('content-length', 0))
    block_size = 1024 # 1 Kibibyte

    # Configuramos la barra de progreso
    progress_bar = tqdm(total=total_size_in_bytes, unit='iB', unit_scale=True)

    with open(nombre_archivo, 'wb') as file:
        for data in response.iter_content(block_size):
            progress_bar.update(len(data))
            file.write(data)
    progress_bar.close()

    if total_size_in_bytes != 0 and progress_bar.n != total_size_in_bytes:
        print("ERROR: Algo salió mal con la descarga.")
    else:
        print(f"\n¡Descarga de {nombre_archivo} completada con éxito!")


# Ejecuto función para descargar carpeta comprimida
print(f"Iniciando descarga de {NOMBRE_ZIP}...")
descargar_con_progreso(URL_COMPLETA, NOMBRE_ZIP)

Iniciando descarga de 0302.zip...


In [ ]:
# Descomprimir (Usamos -q para 'quiet' y evitar imprimir miles de líneas)
print("Descomprimiendo archivos (esto puede tardar unos segundos)...")
!unzip -q -o {NOMBRE_ZIP}
print("¡Descompresión finalizada!")

# Verificar
print("\nContenido de la carpeta:")
!ls {CARPETA_DESTINO} | head -5

Descomprimiendo archivos (esto puede tardar unos segundos)...
¡Descompresión finalizada!

Contenido de la carpeta:
0.txt
1.txt
2.txt
log.txt


# **3. Crear script**
* Parsea cada línea del dataset de Youtube.
* Formato esperado (tabulado): *videoID, uploader, age, category, length, views, ...*
* Indices: 0, 1, 2, 3, 4, 5

Además, en lugar de leer todo asterisco, usamos "[0-9]*.txt". Esto le dice a Spark: "Lee cualquier .txt que empiece con un número (0-9)". Con esto filtramos automáticamente el 'log.txt' y leemos solo 0.txt, 1.txt, etc.

In [ ]:
%%writefile CategoriaDeVideosMasVista.py
import sys
from pyspark import SparkContext

# Inicializamos SparkContext
sc = SparkContext("local", "YoutubeCategoryAnalysis")

# Argumentos: carpeta de entrada y carpeta de salida
ruta_entrada = sys.argv[1]
ruta_salida = sys.argv[2]

def procesar_video(linea):
    try:
        partes = linea.split('\t')

        # Verificamos que la línea tenga suficientes columnas
        if len(partes) > 5:
            categoria = partes[3].strip()
            visitas = int(partes[5].strip())
            return (categoria, visitas)
        else:
            return None
    except:
        return None

try:
    patron_archivos = ruta_entrada + "/[0-9]*.txt"
    datos_rdd = sc.textFile(patron_archivos)

    # Transformaciones:
    # 1. Map: Extraer (Categoria, Visitas)
    # 2. Filter: Quitar líneas corruptas (None)
    # 3. ReduceByKey: Sumar visitas por categoría
    categorias_vistas = datos_rdd.map(procesar_video) \
                                 .filter(lambda x: x is not None) \
                                 .reduceByKey(lambda a, b: a + b)

    categorias_vistas.cache()

    # Encontrar la categoría ganadora para mostrar en consola
    if not categorias_vistas.isEmpty():
        categoria_top = categorias_vistas.max(key=lambda x: x[1])

        print("\n" + "="*50)
        print(f"RESULTADO FINAL: La categoría más vista es '{categoria_top[0]}'")
        print(f"Total de visualizaciones: {categoria_top[1]}")
        print("="*50 + "\n")
    else:
        print("No se encontraron datos válidos.")

    # Guardamos los resultados
    salida_formateada = categorias_vistas.map(lambda x: "{}; {}".format(x[0], x[1]))
    salida_formateada.saveAsTextFile(ruta_salida)

except Exception as e:
    print("Error durante la ejecución:", e)

finally:
    sc.stop()

Writing CategoriaDeVideosMasVista.py


# **4. Ejecutar**

In [ ]:
import os

# Borrar la carpeta de salida de Spark si existe, no la de entrada de datos
!rm -rf resultado_youtube

# Construir el comando spark-submit asegurando que la variable DATASET se interpola correctamente.
# Usamos un f-string para asegurarnos de que {DATASET} se reemplace con su valor ('0302').
spark_submit_command = f"{os.environ['SPARK_HOME']}/bin/spark-submit CategoriaDeVideosMasVista.py {DATASET} resultado_youtube"

# Ejecutar el comando
print(f"Ejecutando: {spark_submit_command}")
!{spark_submit_command}

Ejecutando: /content/spark-3.5.7-bin-hadoop3/bin/spark-submit CategoriaDeVideosMasVista.py 0302 resultado_youtube
26/01/13 14:08:53 INFO SparkContext: Running Spark version 3.5.7
26/01/13 14:08:53 INFO SparkContext: OS info Linux, 6.6.105+, amd64
26/01/13 14:08:53 INFO SparkContext: Java version 17.0.17
26/01/13 14:08:53 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
26/01/13 14:08:53 INFO ResourceUtils: ==============================================================
26/01/13 14:08:53 INFO ResourceUtils: No custom resources configured for spark.driver.
26/01/13 14:08:53 INFO ResourceUtils: ==============================================================
26/01/13 14:08:53 INFO SparkContext: Submitted application: YoutubeCategoryAnalysis
26/01/13 14:08:53 INFO ResourceProfile: Default ResourceProfile created, executor resources: Map(cores -> name: cores, amount: 1, script: , vendor: , memory -> name: memory, amoun

In [ ]:
!head resultado_youtube/*

==> resultado_youtube/part-00000 <==
Entertainment; 19646440
Music; 26922840
Howto & DIY; 932642
People & Blogs; 3648430
Gadgets & Games; 2070193

==> resultado_youtube/part-00001 <==
Sports; 5370975
Film & Animation; 10823098
Comedy; 16626515
Pets & Animals; 2212879
Travel & Places; 305409

==> resultado_youtube/part-00002 <==
News & Politics; 3091377
Autos & Vehicles; 1411860
UNA; 4591983

==> resultado_youtube/_SUCCESS <==
